In [168]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from io import StringIO
#import geopandas as gpd

# Part 2 Data Retrieval
Use the `requests` library and the USGS's API (https://earthquake.usgs.gov/fdsnws/event/1/) to retrieve information about all recorded earthquakes that occurred in Tennessee since 1900.
Answer the following questions:

In [169]:
#a list of all the catalogs available, if needed
cat_url = 'https://earthquake.usgs.gov/fdsnws/event/1/catalogs'

In [170]:
#two catalogs seem useful
baseurl = 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&starttime=1900-01-01&endtime=2020-12-31&catalog='
catalog = ['ushis'
           , 'se'
          ]
tn_eq = pd.DataFrame()
for cat in catalog:
    url = baseurl + cat
    request = requests.get(url)
    temp_df = pd.read_csv(StringIO(request.text))
    tn_eq = pd.concat([temp_df, tn_eq])
    print(url)

https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&starttime=1900-01-01&endtime=2020-12-31&catalog=ushis
https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&starttime=1900-01-01&endtime=2020-12-31&catalog=se


In [172]:
#check
tn_eq.locationSource.unique()

array(['se', 'ushis'], dtype=object)

In [173]:
#check
tn_eq.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2020-10-19T20:19:38.490Z,36.224000,-82.817167,17.27,2.31,md,8.0,70.0,0.175100,0.12,...,2020-10-20T15:44:24.550Z,"6 km N of Greeneville, Tennessee",earthquake,0.91,1.69,0.099,6.0,reviewed,se,se
1,2020-10-18T16:54:12.390Z,36.485833,-81.082000,0.71,-0.61,md,4.0,147.0,0.012360,0.03,...,2020-10-19T13:10:38.350Z,"4 km ESE of Sparta, North Carolina",earthquake,0.34,1.02,0.105,4.0,reviewed,se,se
2,2020-10-17T07:12:44.060Z,35.366833,-85.213500,17.99,2.26,md,16.0,59.0,0.158700,0.14,...,2020-10-17T13:54:48.250Z,"4 km NNW of Flat Top Mountain, Tennessee",earthquake,0.33,0.60,0.152,15.0,reviewed,se,se
3,2020-10-16T05:16:13.960Z,36.510000,-81.084833,5.26,-0.33,md,5.0,272.0,0.007472,0.20,...,2020-10-16T14:15:22.660Z,"3 km E of Sparta, North Carolina",earthquake,1.75,0.75,0.079,4.0,reviewed,se,se
4,2020-10-15T22:56:55.610Z,36.473333,-81.085667,2.34,-0.67,md,4.0,133.0,0.008929,0.05,...,2020-10-16T14:20:04.400Z,"4 km SE of Sparta, North Carolina",earthquake,0.35,0.81,0.063,4.0,reviewed,se,se


In [174]:
#check
tn_eq.shape

(6225, 22)

In [175]:
#find any records that don't have a valid place
tn_eq[tn_eq['place'].isnull()]

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
1656,1961-09-11T02:46:51.700Z,51.4,180.0,60.0,5.9,ms,NaN,NaN,NaN,NaN,...,2018-06-04T20:43:45.000Z,NaN,earthquake,NaN,NaN,NaN,NaN,reviewed,ushis,rot
2601,1946-12-25T11:13:10.000Z,51.5,180.0,90.0,6.5,mb,NaN,NaN,NaN,NaN,...,2018-06-04T20:43:44.000Z,NaN,earthquake,NaN,NaN,NaN,NaN,reviewed,ushis,gr
3647,1911-09-17T03:26:00.000Z,51.0,180.0,NaN,7.1,ms,NaN,NaN,NaN,NaN,...,2018-06-04T20:43:44.000Z,NaN,earthquake,NaN,NaN,NaN,NaN,reviewed,ushis,abe


In [177]:
#and drop
tn_eq = tn_eq.dropna(subset=['place'])
tn_eq.shape

(6222, 22)

In [178]:
#filter for tennessee 'place'
tn_eq = tn_eq[(tn_eq['place'].str.contains('Tenn'))
             | (tn_eq['place'].str.contains('TN'))
             ]
tn_eq.shape

(1077, 22)

In [179]:
#filter for earthquake 'type'
#tn_eq.type.unique()
tn_eq = tn_eq[tn_eq['type'] == 'earthquake']
tn_eq.shape

(1076, 22)

In [185]:
tn_eq.mag.describe()

count    1075.000000
mean        1.901553
std         0.620321
min         0.000000
25%         1.500000
50%         1.900000
75%         2.280000
max         4.600000
Name: mag, dtype: float64